In [1]:
import pandas as pd
import numpy as np
import string

import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('../input/amazonml/train.csv')
test = pd.read_csv('../input/amazonml/test.csv')
tests = test.copy()

In [3]:
print(len(train['Review Text']),len(test['Review Text']))
print(len(train['Review Text'].value_counts()),len(test['Review Text'].value_counts()))

5959 2553
4196 1772


In [4]:
from sklearn.preprocessing import LabelEncoder
le_review = LabelEncoder()

le_review.fit(train['topic'])
train['topic'] = le_review.transform(train['topic'])

In [5]:
train['topic'] = train['topic'].astype(str)

In [6]:
train['review'] = train['Review Text']+' '+train['Review Title']
train.drop(['Review Text','Review Title'],axis=1,inplace=True)

test['review'] = test['Review Text']+' '+test['Review Title']
test.drop(['Review Text','Review Title'],axis=1,inplace=True)

In [7]:
print(len(train))
train = train.drop_duplicates()

print(len(train))

5959
5948


In [8]:
train = train.groupby(['review'],sort=False)['topic'].apply(', '.join).reset_index()

In [9]:
train = pd.concat([train, train['topic'].str.get_dummies(sep=', ')],axis=1)
train.drop(['topic'],axis=1, inplace=True)

In [10]:
train['review'] = train['review'].str.lower()
test['review'] = test['review'].str.lower()

In [11]:
print(len(test))
test = test.groupby(['review'],sort=False).agg({'review':'count'})

2553


In [12]:
test.index.name = None
test = test.reset_index()

test = test.rename(columns={"review": "count", "index": "review"})
print(len(test))

1776


In [13]:
contraction_mapping = {
    "'cause": 'because','â€™': "'",',cause': 'because',';cause': 'because',"ain't": 'am not','ain,t': 'am not','<br />':' ','&quot;':'"',
    'ain;t': 'am not','ain´t': 'am not','ain’t': 'am not',"aren't": 'are not','â€“': '-','â€œ':'"',
    'aren,t': 'are not','aren;t': 'are not','aren´t': 'are not','aren’t': 'are not',"can't": 'cannot',"can't've": 'cannot have','can,t': 'cannot','can,t,ve': 'cannot have',
    'can;t': 'cannot','can;t;ve': 'cannot have',
    'can´t': 'cannot','can´t´ve': 'cannot have','can’t': 'cannot','can’t’ve': 'cannot have',
    "could've": 'could have','could,ve': 'could have','could;ve': 'could have',"couldn't": 'could not',"couldn't've": 'could not have','couldn,t': 'could not','couldn,t,ve': 'could not have','couldn;t': 'could not',
    'couldn;t;ve': 'could not have','couldn´t': 'could not',
    'couldn´t´ve': 'could not have','couldn’t': 'could not','couldn’t’ve': 'could not have','could´ve': 'could have',
    'could’ve': 'could have',"didn't": 'did not','didn,t': 'did not','didn;t': 'did not','didn´t': 'did not',
    'didn’t': 'did not',"doesn't": 'does not','doesn,t': 'does not','doesn;t': 'does not','doesn´t': 'does not',
    'doesn’t': 'does not',"don't": 'do not','don,t': 'do not','don;t': 'do not','don´t': 'do not','don’t': 'do not',
    "hadn't": 'had not',"hadn't've": 'had not have','hadn,t': 'had not','hadn,t,ve': 'had not have','hadn;t': 'had not',
    'hadn;t;ve': 'had not have','hadn´t': 'had not','hadn´t´ve': 'had not have','hadn’t': 'had not','hadn’t’ve': 'had not have',"hasn't": 'has not','hasn,t': 'has not','hasn;t': 'has not','hasn´t': 'has not','hasn’t': 'has not',
    "haven't": 'have not','haven,t': 'have not','haven;t': 'have not','haven´t': 'have not','haven’t': 'have not',"he'd": 'he would',
    "he'd've": 'he would have',"he'll": 'he will',
    "he's": 'he is','he,d': 'he would','he,d,ve': 'he would have','he,ll': 'he will','he,s': 'he is','he;d': 'he would',
    'he;d;ve': 'he would have','he;ll': 'he will','he;s': 'he is','he´d': 'he would','he´d´ve': 'he would have','he´ll': 'he will',
    'he´s': 'he is','he’d': 'he would','he’d’ve': 'he would have','he’ll': 'he will','he’s': 'he is',"how'd": 'how did',"how'll": 'how will',
    "how's": 'how is','how,d': 'how did','how,ll': 'how will','how,s': 'how is','how;d': 'how did','how;ll': 'how will',
    'how;s': 'how is','how´d': 'how did','how´ll': 'how will','how´s': 'how is','how’d': 'how did','how’ll': 'how will',
    'how’s': 'how is',"i'd": 'i would',"i'll": 'i will',"i'm": 'i am',"i've": 'i have','i,d': 'i would','i,ll': 'i will',
    'i,m': 'i am','i,ve': 'i have','i;d': 'i would','i;ll': 'i will','i;m': 'i am','i;ve': 'i have',"isn't": 'is not',
    'isn,t': 'is not','isn;t': 'is not','isn´t': 'is not','isn’t': 'is not',"it'd": 'it would',"it'll": 'it will',"It's":'it is',
    "it's": 'it is','it,d': 'it would','it,ll': 'it will','it,s': 'it is','it;d': 'it would','it;ll': 'it will','it;s': 'it is','it´d': 'it would','it´ll': 'it will','it´s': 'it is',
    'it’d': 'it would','it’ll': 'it will','it’s': 'it is',
    'i´d': 'i would','i´ll': 'i will','i´m': 'i am','i´ve': 'i have','i’d': 'i would','i’ll': 'i will','i’m': 'i am',
    'i’ve': 'i have',"let's": 'let us','let,s': 'let us','let;s': 'let us','let´s': 'let us',
    'let’s': 'let us',"ma'am": 'madam','ma,am': 'madam','ma;am': 'madam',"mayn't": 'may not','mayn,t': 'may not','mayn;t': 'may not',
    'mayn´t': 'may not','mayn’t': 'may not','ma´am': 'madam','ma’am': 'madam',"might've": 'might have','might,ve': 'might have','might;ve': 'might have',"mightn't": 'might not','mightn,t': 'might not','mightn;t': 'might not','mightn´t': 'might not',
    'mightn’t': 'might not','might´ve': 'might have','might’ve': 'might have',"must've": 'must have','must,ve': 'must have','must;ve': 'must have',
    "mustn't": 'must not','mustn,t': 'must not','mustn;t': 'must not','mustn´t': 'must not','mustn’t': 'must not','must´ve': 'must have',
    'must’ve': 'must have',"needn't": 'need not','needn,t': 'need not','needn;t': 'need not','needn´t': 'need not','needn’t': 'need not',"oughtn't": 'ought not','oughtn,t': 'ought not','oughtn;t': 'ought not',
    'oughtn´t': 'ought not','oughtn’t': 'ought not',"sha'n't": 'shall not','sha,n,t': 'shall not','sha;n;t': 'shall not',"shan't": 'shall not',
    'shan,t': 'shall not','shan;t': 'shall not','shan´t': 'shall not','shan’t': 'shall not','sha´n´t': 'shall not','sha’n’t': 'shall not',
    "she'd": 'she would',"she'll": 'she will',"she's": 'she is','she,d': 'she would','she,ll': 'she will',
    'she,s': 'she is','she;d': 'she would','she;ll': 'she will','she;s': 'she is','she´d': 'she would','she´ll': 'she will',
    'she´s': 'she is','she’d': 'she would','she’ll': 'she will','she’s': 'she is',"should've": 'should have','should,ve': 'should have','should;ve': 'should have',
    "shouldn't": 'should not','shouldn,t': 'should not','shouldn;t': 'should not','shouldn´t': 'should not','shouldn’t': 'should not','should´ve': 'should have',
    'should’ve': 'should have',"that'd": 'that would',"that's": 'that is','that,d': 'that would','that,s': 'that is','that;d': 'that would',
    'that;s': 'that is','that´d': 'that would','that´s': 'that is','that’d': 'that would','that’s': 'that is',"there'd": 'there had',
    "there's": 'there is','there,d': 'there had','there,s': 'there is','there;d': 'there had','there;s': 'there is',
    'there´d': 'there had','there´s': 'there is','there’d': 'there had','there’s': 'there is',
    "they'd": 'they would',"they'll": 'they will',"they're": 'they are',"they've": 'they have',
    'they,d': 'they would','they,ll': 'they will','they,re': 'they are','they,ve': 'they have','they;d': 'they would','they;ll': 'they will','they;re': 'they are',
    'they;ve': 'they have','they´d': 'they would','they´ll': 'they will','they´re': 'they are','they´ve': 'they have','they’d': 'they would','they’ll': 'they will',
    'they’re': 'they are','they’ve': 'they have',"wasn't": 'was not','wasn,t': 'was not','wasn;t': 'was not','wasn´t': 'was not',
    'wasn’t': 'was not',"we'd": 'we would',"we'll": 'we will',"we're": 'we are',"we've": 'we have','we,d': 'we would','we,ll': 'we will',
    'we,re': 'we are','we,ve': 'we have','we;d': 'we would','we;ll': 'we will','we;re': 'we are','we;ve': 'we have',
    "weren't": 'were not','weren,t': 'were not','weren;t': 'were not','weren´t': 'were not','weren’t': 'were not','we´d': 'we would','we´ll': 'we will',
    'we´re': 'we are','we´ve': 'we have','we’d': 'we would','we’ll': 'we will','we’re': 'we are','we’ve': 'we have',"what'll": 'what will',"what're": 'what are',"what's": 'what is',
    "what've": 'what have','what,ll': 'what will','what,re': 'what are','what,s': 'what is','what,ve': 'what have','what;ll': 'what will','what;re': 'what are',
    'what;s': 'what is','what;ve': 'what have','what´ll': 'what will',
    'what´re': 'what are','what´s': 'what is','what´ve': 'what have','what’ll': 'what will','what’re': 'what are','what’s': 'what is',
    'what’ve': 'what have',"where'd": 'where did',"where's": 'where is','where,d': 'where did','where,s': 'where is','where;d': 'where did',
    'where;s': 'where is','where´d': 'where did','where´s': 'where is','where’d': 'where did','where’s': 'where is',
    "who'll": 'who will',"who's": 'who is','who,ll': 'who will','who,s': 'who is','who;ll': 'who will','who;s': 'who is',
    'who´ll': 'who will','who´s': 'who is','who’ll': 'who will','who’s': 'who is',"won't": 'will not','won,t': 'will not','won;t': 'will not',
    'won´t': 'will not','won’t': 'will not',"wouldn't": 'would not','wouldn,t': 'would not','wouldn;t': 'would not','wouldn´t': 'would not',
    'wouldn’t': 'would not',"you'd": 'you would',"you'll": 'you will',"you're": 'you are','you,d': 'you would','you,ll': 'you will',
    'you,re': 'you are','you;d': 'you would','you;ll': 'you will',
    'you;re': 'you are','you´d': 'you would','you´ll': 'you will','you´re': 'you are','you’d': 'you would','you’ll': 'you will','you’re': 'you are',
    '´cause': 'because','’cause': 'because',"you've": "you have","could'nt": 'could not',
    "havn't": 'have not',"here’s": "here is",'i""m': 'i am',"i'am": 'i am',"i'l": "i will","i'v": 'i have',"wan't": 'want',"was'nt": "was not","who'd": "who would",
    "who're": "who are","who've": "who have","why'd": "why would","would've": "would have","y'all": "you all","y'know": "you know","you.i": "you i",
    "your'e": "you are","arn't": "are not","agains't": "against","c'mon": "common","doens't": "does not",'don""t': "do not","dosen't": "does not",
    "dosn't": "does not","shoudn't": "should not","that'll": "that will","there'll": "there will","there're": "there are",
    "this'll": "this all","u're": "you are", "ya'll": "you all","you'r": "you are","you’ve": "you have","d'int": "did not","did'nt": "did not","din't": "did not","dont't": "do not","gov't": "government",
    "i'ma": "i am","is'nt": "is not","‘I":'I',' :( ':' sad '}

In [14]:
def correct_contraction(x, dic):
    for word in dic.keys():
        if word in x:
            x = x.replace(word, dic[word])
    return x

In [15]:
train['review'] = train['review'].apply(lambda x: correct_contraction(x, contraction_mapping))
test['review']  = test['review'].apply(lambda x: correct_contraction(x, contraction_mapping))

In [16]:
bad_case_words = {' solimo ':' brand ',' naturewise ':' nature wise ','natrue':'nature',' sleep3 ':' sleep ',' grosssssss ': ' gross ',
                  ' revly ': ' brand ',' asorbic ': ' ascorbic ','xssential':'brand','xssentail':'brand','🤢':' nauseated ','😔':'pensive',
                  '🙁':'frowning','😉':'winkling','😞':'disappointed',' mykind ':' brand ',' lasership ': ' company ','😡':'angry',
                  '\U0001f92e':'vomiting','⚠':'danger'}

In [17]:

all_punct = list(string.punctuation)

In [18]:
special_punc_mappings = {"—": "-", "–": "-", "_": "-", '”': '"', "″": '"', '“': '"', '•': '.', '−': '-',
                         "’": "'", "‘": "'", "´": "'", "`": "'", '\u200b': ' ', '\xa0': ' ','،':'','„':'',
                         '…': ' ... ', '\ufeff': ''}

In [19]:
def clean_bad_case_words(text):
    for bad_word in bad_case_words:
        if bad_word in text:
            text = text.replace(bad_word, bad_case_words[bad_word])
    return text

def clean_special_punctuations(text):
    for punc in special_punc_mappings:
        if punc in text:
            text = text.replace(punc, special_punc_mappings[punc])
    # remove_diacritics don´t' ->  'don t'
    #text = remove_diacritics(text)
    return text

def spacing_punctuation(text):
    """
    add space before and after punctuation and symbols
    """
    for punc in all_punct:
        if punc in text:
            text = text.replace(punc, f' {punc} ')
    return text

In [20]:
def preprocess(text):
    text = clean_special_punctuations(text)
    text = spacing_punctuation(text)
    text = clean_bad_case_words(text)
    return text

In [21]:
train["review"] = train["review"].apply(preprocess)
test["review"] = test["review"].apply(preprocess)

In [22]:
train['review'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
test['review'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

In [23]:
train['review'] = train['review'].str.replace(" '",' ').str.replace(' s ','')
test['review'] = test['review'].str.replace(" '",'').str.replace(' s ','')

In [24]:
train.to_csv('train1.csv',index=False)
test.to_csv('test1.csv',index=False)

In [25]:
from fastai import *
from fastai.text import *
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train, test_size=0.1)

In [26]:
label_cols = ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17','18', '19', '2', '20', '3', '4', '5', '6', '7', '8', '9']

In [27]:
MODEL_PATH = ''
TEXT_COL = 'review'

data_lm = TextClasDataBunch.from_df(MODEL_PATH,train_df,valid_df=val_df, test_df=test, text_cols=[TEXT_COL], label_cols=label_cols)

In [28]:
awd_lstm_clas_config = dict(emb_sz=400, n_hid=1150, n_layers=3, pad_token=1, qrnn=False, bidir=False, output_p=0.4,
                       hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5)

In [29]:
learner = text_classifier_learner(data_lm, AWD_LSTM, max_len=100,config=awd_lstm_clas_config, pretrained = False)

In [30]:
fnames = ['../input/awd-lstm/lstm_wt103.pth','../input/awd-lstm/itos_wt103.pkl']
learner.load_pretrained(*fnames, strict=False)
learner.freeze()

In [31]:
learner.unfreeze()
learner.fit_one_cycle(2, 2e-3)

epoch,train_loss,valid_loss,time
0,0.641571,0.558149,00:07
1,0.457231,0.369388,00:07


In [32]:
def get_preds_as_nparray(ds_type) -> np.ndarray:
    """
    the get_preds method does not yield the elements in order by default
    we borrow the code from the RNNLearner to resort the elements into their correct order
    """
    preds = learner.get_preds(ds_type)[0].detach().cpu().numpy()
    #y = learner.get_preds(ds_type)[1].detach().cpu().numpy()
    
    sampler = [i for i in data_lm.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    
    return preds[reverse_sampler, :]

In [33]:
preds = get_preds_as_nparray(DatasetType.Test)
len(test)

1776

In [34]:
dataset = pd.DataFrame({0:preds[:,0], 1:preds[:,1], 10:preds[:,2], 11:preds[:,3], 12:preds[:,4], 13:preds[:,5], 14:preds[:,6], 
                        15:preds[:,7], 16:preds[:,8], 17:preds[:,9],18:preds[:,10], 19:preds[:,11], 2:preds[:,12], 20:preds[:,12],
                        3:preds[:,14], 4:preds[:,15], 5:preds[:,16], 6:preds[:,17], 7:preds[:,18] ,8:preds[:,19] , 9:preds[:,20]})


In [35]:
dataset.head(2)

,0,1,10,11,12,13,14,15,16,17,...,19,2,20,3,4,5,6,7,8,9
0,0.256656,0.351148,0.237631,0.351388,0.278126,0.291281,0.510873,0.199369,0.305037,0.369061,...,0.288414,0.322546,0.322546,0.275172,0.274005,0.250349,0.304663,0.252855,0.237475,0.215506
1,0.266111,0.176097,0.264294,0.657040,0.261072,0.307886,0.405986,0.236624,0.253040,0.230334,...,0.223259,0.222224,0.222224,0.237521,0.235706,0.237652,0.233382,0.232298,0.199434,0.244199


In [36]:
x=pd.DataFrame(dataset).T
rslt = pd.DataFrame(np.zeros((0,6)), columns=['top1','top2','top3','top4','top5','top6'])
for i in x.columns:
    df1row = pd.DataFrame(x.nlargest(6, i).index.tolist(), index=['top1','top2','top3','top4','top5','top6']).T
    rslt = pd.concat([rslt, df1row], axis=0)

In [37]:
rslt = rslt.reset_index()
rslt.drop(['index'],axis=1,inplace=True)

In [38]:
final = pd.merge(test,rslt,left_index=True,right_index=True,how='inner')

In [39]:
def traildecis(df):
    df = df.astype(str)
    df = df.astype(str).replace('\.0', '', regex=True)
    return df

In [40]:
final['top1'] = traildecis(final['top1'])
final['top2'] = traildecis(final['top2'])
final['top3'] = traildecis(final['top3'])
final['top4'] = traildecis(final['top4'])
final['top5'] = traildecis(final['top5'])
final['top6'] = traildecis(final['top6'])

In [41]:
final.head()

,review,count,top1,top2,top3,top4,top5,top6
0,i use chia seed in my protein shakes . these ...,2,14,17,11,1,2,20
1,do not waste your money . no change . no res...,1,11,14,13,0,10,12
2,i use the book fortify your life by tieraona...,2,11,14,17,2,20,19
3,i used to be loyal customer to this brand . i...,4,0,17,12,19,2,20
4,i have not received it yet . shipping,1,15,12,0,11,17,6


In [42]:
def merging(data):
    beep =[]
    for index, row in data.iterrows():
        l = row["count"]
    
        if(l==1):
            row['top1'] = row['top1']
    
        elif(l==2):
            row['top1'] = row['top1']+','+row['top2']
        
        elif(l==3):
            row['top1'] = row['top1']+','+row['top2']+','+row['top3']
    
        elif(l==4):
            row['top1'] = row['top1']+','+row['top2']+','+row['top3']+','+row['top4']
        
        elif(l==5):
            row['top1'] = row['top1']+','+row['top2']+','+row['top3']+','+row['top4']+','+row['top5']
        
        elif(l==6):
            row['top1'] = row['top1']+','+row['top2']+','+row['top3']+','+row['top4']+','+row['top5']+','+row['top6']
            
        beep.append(row['top1'])
    
    data['new'] = beep
    
    return data
        

In [43]:
final = merging(final)
final.drop(['top1','top2','top3','top4','top5','top6'],axis=1,inplace=True)

In [44]:
new_final = pd.DataFrame(final.new.str.split(',').tolist(), index=final.review).stack()
new_final = new_final.reset_index([0, 'review'])
new_final.columns = ['review', 'topic']

In [45]:
sub_test = pd.merge(tests,new_final,left_index=True,right_index=True,how='inner')
sub_test.drop(['review'],axis=1,inplace=True)

In [46]:
len(sub_test)

2553

In [47]:
sub_test['topic'] = sub_test['topic'].astype(int)
sub_test['topic'] = le_review.inverse_transform(sub_test['topic'])
sub_test.to_csv('submission_amazon.csv',index=False)

In [48]:
sub_test['topic'].value_counts()

Bad Taste/Flavor          543
Packaging                 491
Not Effective             435
Quality/Contaminated      349
Texture                   338
Shipment and delivery     196
Allergic                  111
Color and texture          59
Wrong Product received     13
Too big to swallow          9
Pricing                     3
Too Sweet                   3
Ingredients                 2
Smells Bad                  1
Name: topic, dtype: int64